In [1]:
!pip install ta
!pip install scikit-learn scikit-optimize xgboost lightgbm catboost

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29411 sha256=94f275ad5ee5d6d939674da5f68336c7474d4e6e988d16c897cbd99140e433eb
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.8 MB/s eta 0:00:00


In [2]:
import ast
symbol =  "CADJPY"
file_result = "CADJPY case study1.txt"

with open(file_result, "r") as f:
  result_casestudy = f.readlines()

diff_or_pct = 1
upper_rate, lower_rate = 70,30

profit_case_month_1= {}
profit_case_month_2 = {}

predict_case_month_1 = {}
predict_case_month_2 = {}

parameter = {}

for i, line in enumerate(result_casestudy):
    line = line.replace('\n', '')

    if "parameter" in line:
      # Xử lý chuỗi để chuyển đổi thành từ điển

      para_LGBM = result_casestudy[i+1].split(":")[1].strip()
      dict_string = para_LGBM.replace("OrderedDict", "")
      parameter_dict = ast.literal_eval(dict_string)
      parameter["LGBM"] = parameter_dict

      para_LGBM2= result_casestudy[i+2].split(":")[1].strip()
      dict_string = para_LGBM2.replace("OrderedDict", "")
      parameter_dict = ast.literal_eval(dict_string)
      parameter["LGBM2"] = parameter_dict

      para_XGB = result_casestudy[i+3].split(":")[1].strip()
      dict_string = para_XGB.replace("OrderedDict", "")
      parameter_dict = ast.literal_eval(dict_string)
      parameter["XGB"] = parameter_dict


      para_RF = result_casestudy[i+4].split(":")[1].strip()
      dict_string = para_RF.replace("OrderedDict", "")
      parameter_dict = ast.literal_eval(dict_string)
      parameter["RF"] = parameter_dict


In [8]:
import requests
import pandas as pd
from datetime import datetime
import os

data= pd.read_csv(f'{symbol}.csv')
data.head()

,time,Open,High,Low,Close,Volume
0,2012-01-01,75.295,75.697,75.255,75.649,603
1,2012-01-02,75.418,75.707,75.271,75.411,26982
2,2012-01-03,75.431,76.189,75.224,75.920,64692
3,2012-01-04,75.848,75.987,75.536,75.726,151645
4,2012-01-05,75.659,75.901,75.338,75.590,183422


In [9]:

# Chuyển cột 'time' thành index và đổi tên index thành 'Date'
data.set_index('time', inplace=True)
data.index.name = 'Date'

# Chuyển index sang định dạng datetime để dễ lọc
data.index = pd.to_datetime(data.index)

# Lọc dữ liệu từ năm 2019 trở đi
data= data[data.index.year >= 2019]

# Hiển thị DataFrame đã lọc
data


,Open,High,Low,Close,Volume
Date,,,,,
2019-01-02,80.348,80.626,79.648,80.182,218284
2019-01-03,80.159,80.331,76.699,79.810,149023
2019-01-04,79.799,81.122,79.700,81.111,123147
2019-01-07,81.051,81.829,80.851,81.732,214449
2019-01-08,81.735,82.113,81.412,81.907,79407
...,...,...,...,...,...
2024-05-02,112.436,113.803,111.909,112.294,154878
2024-05-03,112.313,112.403,111.553,111.688,140785
2024-05-06,111.720,112.779,111.619,112.596,93838


In [10]:
import pandas as pd
import ta
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy


data_ori  = data.rename(columns = {"Date":"DATE", "Open": "OPEN", "High": "HIGH", "Low": "LOW","Close": "CLOSE", "Volume": "VOLUME"  })

# Gán các cột cho dễ nhìn hơn
data_ori = ta.add_all_ta_features(
    df=data_ori,
    open="OPEN",
    high="HIGH",
    low="LOW",
    close="CLOSE",
    volume="VOLUME",
    fillna=True
)

In [17]:
data_ori.tail()

,OPEN,HIGH,LOW,CLOSE,VOLUME,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
Date,,,,,,,,,,,,,,,,,,,,,
2024-05-02,112.436,113.803,111.909,112.294,154878,8.593358e+06,3727847,0.377734,-20532.842391,-204.223970,...,0.513389,0.564762,-0.051373,2.494399,0.666909,1.827490,114.031172,-0.109414,-0.109474,40.048889
2024-05-03,112.313,112.403,111.553,111.688,140785,8.497293e+06,3587062,0.316304,-29787.537763,-530.099087,...,0.368304,0.525470,-0.157167,2.942097,1.121947,1.820150,113.993968,-0.539655,-0.541116,39.293108
2024-05-06,111.720,112.779,111.619,112.596,93838,8.561524e+06,3680900,0.304526,-13360.046083,273.194228,...,0.314533,0.483283,-0.168750,0.359877,0.969533,-0.609656,113.981163,0.812979,0.809692,40.425532
2024-05-07,112.596,113.059,112.500,112.682,103531,8.525408e+06,3784431,0.273783,-10179.515785,313.432209,...,0.274901,0.441606,-0.166705,-1.112986,0.553029,-1.666015,113.964572,0.076379,0.076350,40.532788
2024-05-08,112.684,113.046,112.416,112.968,52884,8.565197e+06,3837315,0.260009,-6564.610102,-57.777400,...,0.260936,0.405472,-0.144536,-5.761246,-0.709826,-5.051420,113.955993,0.253812,0.253490,40.889476


In [16]:
import numpy as np
# data = data_ori.copy()
# data['Date'] = pd.to_datetime(data['Date'])
# # Giả sử 'data' là DataFrame của bạn chứa cột 'CLOSE'
data.reset_index(inplace=True)
time_test = 2
time_split = data['Date'].max() - pd.DateOffset(months=time_test)
data_smooth = data[data['Date'] <= time_split].copy()
# data_smooth['CLOSE_MA'] = data_smooth['CLOSE'].rolling(window=5).mean()
"""Chổ này là chổ cần được chỉnh sửa lại theo pct_change hay là diff"""
if diff_or_pct == 1:

  data_smooth['pct_change'] = data_smooth['CLOSE'].diff()
if diff_or_pct ==2:
  data_smooth['pct_change'] = data_smooth['CLOSE'].pct_change()

#pct change là tính theo phần trăm lợi nhuận
"""-----------------------------------------------------------------"""

# CHIA Theo phần trăm là xaidf pct_change, thứ hai là chia theo chênh lệch diff()
# 10 +12 12-10/10 =0.2 này pct_change(),  10  ngày 12 thì lời 2 đồng này là  diff()
# 100 102 2%
data_smooth['pct_change_ma'] = data_smooth['pct_change'].rolling(window=1).mean()

data_smooth['pct_change_ma'] = data_smooth['pct_change_ma'].shift(-1)
data_smooth = data_smooth.dropna()
# Giả sử data đã có cột 'pct_change'
# Xác định ngưỡng dựa trên phân phối hoặc độ lệch chuẩn


"""Chổ này là chổ cần chỉnh sửa theo tỷ lệ hiện ta có hai tỷ lệ là 65 35 và    60 40"""
threshold_upper = np.percentile(data_smooth['pct_change_ma'].dropna(), upper_rate)
threshold_lower = np.percentile(data_smooth['pct_change_ma'].dropna(), lower_rate) #quan tâm số dưới


""" -------------------------------------------------------------------------------- """

 # Nhãn là 1 0 -1 với 1 là tăng, 0 là không làm gì hết và -1 là giảm.


# Gán nhãn dựa trên ngưỡng

data_smooth['label'] = np.select(
    [
        data_smooth['pct_change_ma'] <= threshold_lower,
        data_smooth['pct_change_ma'] >= threshold_upper
    ],
    [
        -1,  # Giảm
        1   # Tăng
    ],
    default=0  # Ổn định
)

print(data_smooth['label'].value_counts(normalize=True))
print(f"Train size: {data_smooth.shape}")


# data_smooth.reset_index(inplace=True)
# data_smooth = data_smooth.drop(['pct_change', 'pct_change_ma'], axis=1)
data_smooth = data_smooth.dropna()
scaler = StandardScaler()

# Tách dữ liệu thành tập train-validation và tập test
# Giả sử 'Date' là cột chứa ngày tháng và định dạng YYYY-MM-DD
time_eval = 2

eval_months = data_smooth['Date'].max() - pd.DateOffset(months=time_eval)
#  xác định thời điểm chia eval và test

eval_data = data_smooth[data_smooth['Date'] >= eval_months].copy()

train_data = data_smooth[data_smooth['Date'] < eval_months].copy()
#  dữ liệu eval là 1 tháng hiện tại, dùng để xác định xem parameters đã okie chưa


# y_train_lstm = train_data['pct_change'].copy()
# y_val_lstm = eval_data['pct_change'].copy()
train_data = train_data.drop(['pct_change', 'pct_change_ma'], axis=1)

eval_data = eval_data.drop(['pct_change', 'pct_change_ma'], axis=1)

train_data_scaled = pd.DataFrame(scaler.fit_transform(train_data.drop(['Date','label'],axis=1)), columns=train_data.drop(['Date','label'],axis=1).columns)
# dùng train để fit_transform và sau đó thì transform eval và test chứ ko dùng fit_transform
#  điều này đảm tính phân phối dữ liệu và cũng không làm leak data future
eval_data_scaled = pd.DataFrame(scaler.transform(eval_data.drop(['Date','label'],axis=1)), columns=eval_data.drop(['Date','label'],axis=1).columns)


#  gắn DATE và label vào lại data đã scaled để dễ triển khai
train_data_scaled['Date'] = train_data['Date'].to_numpy()
train_data_scaled['label'] = train_data['label'].to_numpy()

eval_data_scaled['Date'] = eval_data['Date'].to_numpy()
eval_data_scaled['label'] = eval_data['label'].to_numpy()


#  bỏ DATE và label để hình thành features, lấy label làm test
if 'DATE' in train_data.columns:
    train_data_features = train_data_scaled.drop(['Date', 'label'], axis=1)
    val_data_features = eval_data_scaled.drop(['Date', 'label'], axis=1)

else:
    train_data_features = train_data_scaled.drop(['label'], axis=1)
    val_data_features = eval_data_scaled.drop(['label'], axis=1)
    # test_data_features = test_data_scaled.drop(['label'], axis=1)

# Tách nhãn (y)
train_labels = train_data_scaled['label']
val_labels = eval_data_scaled['label']


# Xác định X và y cho tập train, validation và test
X_train = train_data_features
y_train = train_labels

X_val = val_data_features
y_val = val_labels



# Hiển thị kích thước của các tập để kiểm tra
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")



# Hiển thị kích thước của các tập dữ liệu
print(f"Train size: {train_data.shape}")
print(f"Validation size: {eval_data.shape}")

label
 0    0.399106
-1    0.300819
 1    0.300074
Name: proportion, dtype: float64
Train size: (1343, 95)
X_train shape: (1299, 92), y_train shape: (1299,)
X_val shape: (44, 92), y_val shape: (44,)
Train size: (1299, 93)
Validation size: (44, 93)


In [18]:
def calculate_total_profit(y_pred, prices):
    """
    Tính tổng lợi nhuận dựa trên dự đoán của meta-model.

    :param y_true: Nhãn thực tế (không nhất thiết sử dụng trong hàm này nhưng để cho mục đích so sánh).
    :param y_pred: Dự đoán từ meta-model.
    :param prices: Mảng giá tại mỗi điểm thời gian (ví dụ, giá đóng cửa của cổ phiếu).
    :return: Tổng lợi nhuận từ chiến lược.
    """
    profit = 0
    for i in range(1, len(prices)):
        if y_pred[i-1] == 1:  # Dự đoán là tăng
            profit += prices[i] - prices[i-1]  # Lợi nhuận từ việc mua và sau đó bán
        elif y_pred[i-1] == -1:  # Dự đoán là giảm
            profit += prices[i-1] - prices[i]  # Lợi nhuận từ việc bán và sau đó mua lại
    return profit

def caculate_profit(sym, total_profit):
  # Khởi tạo các biến để tính tiền cho forex
  usd_rate = {"EURCHF":1.11, "EURGBP":1.22, "GBPAUD": 0.64, "EURJPY": 0.0066, \
                "USDCHF":1.11, "EURNZD":0.59,  'AUDJPY':0.0066, 'EURCAD':0.72, 'NZDUSD':1,\
                 'AUDCAD':0.72, 'NZDJPY': 0.0066, 'GBPJPY':0.0066, 'USDJPY':0.0066, 'AUDNZD':0.59\
                 , 'USDCAD':0.72 }
  volume = 0.01
  contract_size = 100000
# Sau đó tính tiền
  profit = total_profit*volume*contract_size*usd_rate[sym]
  return profit


In [19]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.metrics import accuracy_score
import numpy as np
np.int = int


In [26]:
# X_train =X_train.drop(["Date"], axis =1)

X_val = X_val.drop(["Date"], axis =1)

# LGBM

In [31]:
import numpy

from lightgbm import LGBMClassifier
split_point_CV = int(len(X_train)*0.90)
X_train_BY = X_train[split_point_CV:]
y_train_BY = y_train[split_point_CV:]

# Mô hình LightGBM
lgbm_model = LGBMClassifier()
# Thay parameter vào model
para_lgbm =  parameter["LGBM"]
para_lgbm_dict = dict(para_lgbm)
LGBMbase = LGBMClassifier(**para_lgbm_dict)
LGBMbase.fit(X_train,y_train)

y_pred = LGBMbase.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
prices = X_val['CLOSE'].copy().to_numpy()
total_profit = calculate_total_profit(y_pred,prices)
print(total_profit)


data_make_test = data[data['Date'] > time_split].copy()
test_months_ql = data_make_test['Date'].min() + pd.DateOffset(months=0)
end_months = data_make_test['Date'].min() + pd.DateOffset(months=12)
test_data = data_make_test[(data_make_test['Date'] > test_months_ql) & (data_make_test['Date']<= end_months)].copy()
X_test = pd.DataFrame(scaler.transform(test_data.drop(['Date'],axis=1)), columns=test_data.drop(['Date'],axis=1).columns)

X_test['Date'] = test_data['Date'].to_numpy()

X_test.set_index('Date', inplace=True)
print(f"X_test shape: {X_test.shape}")
print(f"Test size: {test_data.shape}")
y_pred = LGBMbase.predict(X_test)
prices = X_test['CLOSE'].copy().to_numpy()
total_profit = calculate_total_profit(y_pred,prices)
print(total_profit)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002123 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20986
[LightGBM] [Info] Number of data points in the train set: 1299, number of used features: 87
[LightGBM] [Info] Start training from score -1.200642
[LightGBM] [Info] Start training from score -0.919375
[LightGBM] [Info] Start training from score -1.203203
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

# LGBM2

In [32]:
import numpy
from lightgbm import LGBMClassifier
split_point_CV = int(len(X_train)*0.80)
X_train_BY = X_train[split_point_CV:]
y_train_BY = y_train[split_point_CV:]

# Mô hình LightGBM
lgbm_model = LGBMClassifier()
# đưa para vào model LGBM2
para_lgbm2 = parameter["LGBM2"]
para_lgbm2_dict = dict(para_lgbm2)
LGBMbase2 = LGBMClassifier(**para_lgbm2_dict)
LGBMbase2.fit(X_train,y_train)

y_pred = LGBMbase2.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
prices = X_val['CLOSE'].copy().to_numpy()
total_profit = calculate_total_profit(y_pred,prices)
print(total_profit)


data_make_test = data[data['Date'] > time_split].copy()
test_months_ql = data_make_test['Date'].min() + pd.DateOffset(months=0)
end_months = data_make_test['Date'].min() + pd.DateOffset(months=12)
test_data = data_make_test[(data_make_test['Date'] > test_months_ql) & (data_make_test['Date']<= end_months)].copy()
X_test = pd.DataFrame(scaler.transform(test_data.drop(['Date'],axis=1)), columns=test_data.drop(['Date'],axis=1).columns)

X_test['Date'] = test_data['Date'].to_numpy()

X_test.set_index('Date', inplace=True)
print(f"X_test shape: {X_test.shape}")
print(f"Test size: {test_data.shape}")
y_pred = LGBMbase.predict(X_test)
prices = X_test['CLOSE'].copy().to_numpy()
total_profit = calculate_total_profit(y_pred,prices)
print(total_profit)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002571 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20983
[LightGBM] [Info] Number of data points in the train set: 1299, number of used features: 86
[LightGBM] [Info] Start training from score -1.200642
[LightGBM] [Info] Start training from score -0.919375
[LightGBM] [Info] Start training from score -1.203203
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

# XGB Model

In [36]:

y_train_012 = le.fit_transform(y_train)

split_point_CV = int(len(X_train)*0.90)
X_train_BY = X_train[split_point_CV:]
y_train_BY = y_train_012[split_point_CV:]

np.int = int


# tìm được paras tốt nhất thì huấn luyện với đầy đủ data
import tensorflow as tf

para_xgb_dict = dict(parameter["XGB"])
xgbbase = XGBClassifier(**para_xgb_dict)

xgbbase.fit(X_train,y_train_012, verbose=True)


# kiếm tra model đã huấn luyện full data có lời trên tập eval không
y_pred = xgbbase.predict(X_val)
y_pred = le.inverse_transform(y_pred)
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
prices = X_val['CLOSE'].copy().to_numpy()
total_profit = calculate_total_profit(y_pred,prices)
print(total_profit)

data_make_test = data[data['Date'] > time_split].copy()
test_months_ql = data_make_test['Date'].min() + pd.DateOffset(months=0)
end_months = data_make_test['Date'].min() + pd.DateOffset(months=12)
test_data = data_make_test[(data_make_test['Date'] > test_months_ql) & (data_make_test['Date']<= end_months)].copy()
X_test = pd.DataFrame(scaler.transform(test_data.drop(['Date'],axis=1)), columns=test_data.drop(['Date'],axis=1).columns)

X_test['Date'] = test_data['Date'].to_numpy()

X_test.set_index('Date', inplace=True)
print(f"X_test shape: {X_test.shape}")
print(f"Test size: {test_data.shape}")
y_pred = LGBMbase.predict(X_test)
prices = X_test['CLOSE'].copy().to_numpy()
total_profit = calculate_total_profit(y_pred,prices)
print("lợi nhuận tap tes:" ,total_profit)

Accuracy: 29.55%
-0.0006571211590205017
X_test shape: (42, 91)
Test size: (42, 92)
lợi nhuận tap tes: -0.09650293592482129


# RF model

In [34]:
from sklearn.ensemble import RandomForestClassifier

rf_xgb_dict = dict(parameter["RF"])
RFbase = RandomForestClassifier(**rf_xgb_dict)
# Thêm các para vào model
RFbase.fit(X_train,y_train)
y_pred = RFbase.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
prices = X_val['CLOSE'].copy().to_numpy()
total_profit = calculate_total_profit(y_pred,prices)
print(total_profit)
data_make_test = data[data['Date'] > time_split].copy()
test_months_ql = data_make_test['Date'].min() + pd.DateOffset(months=0)
end_months = data_make_test['Date'].min() + pd.DateOffset(months=12)
test_data = data_make_test[(data_make_test['Date'] > test_months_ql) & (data_make_test['Date']<= end_months)].copy()
X_test = pd.DataFrame(scaler.transform(test_data.drop(['Date'],axis=1)), columns=test_data.drop(['Date'],axis=1).columns)

X_test['Date'] = test_data['Date'].to_numpy()

X_test.set_index('Date', inplace=True)
print(f"X_test shape: {X_test.shape}")
print(f"Test size: {test_data.shape}")
y_pred = LGBMbase.predict(X_test)
prices = X_test['CLOSE'].copy().to_numpy()
total_profit = calculate_total_profit(y_pred,prices)
print(total_profit)

Accuracy: 27.27%
-0.16719039774524513
X_test shape: (42, 91)
Test size: (42, 92)
-0.09650293592482129


# Các base model vào dùng để stacking model

In [37]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.base import clone
kfold = KFold(n_splits=3, shuffle=True, random_state=42)

base_models = [
    xgbbase,
    RFbase,
    LGBMbase,
    LGBMbase2,
]


meta_features = np.zeros((X_train.shape[0], len(base_models)))
meta_target = np.zeros(X_train.shape[0])


for i, model in enumerate(base_models):
    for train_index, holdout_index in kfold.split(X_train, y_train):
        # Clone model để không làm thay đổi model gốc
        instance = clone(model)
        if isinstance(instance, XGBClassifier):
            y_train_index = le.fit_transform(y_train.copy().iloc[train_index])
        else:
            y_train_index = y_train.copy().iloc[train_index]
        # Huấn luyện mỗi base model trên phần train của fold
        instance.fit(X_train.iloc[train_index], y_train_index)

        # Dự đoán trên phần hold-out và lưu vào meta_features

        y_pred = instance.predict(X_train.iloc[holdout_index])
        if isinstance(instance, XGBClassifier):
            y_pred = le.inverse_transform(y_pred)
        meta_features[holdout_index, i] = y_pred

meta_target = y_train


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with

# Meta model được train bằng CAT, random forest, MLP classifiers

In [38]:
from catboost import CatBoostClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

catboost_meta_model = CatBoostClassifier(iterations=100, learning_rate=0.01, depth=10, verbose=0)
catboost_meta_model.fit(meta_features, meta_target)


random_forest_meta_model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)
random_forest_meta_model.fit(meta_features, meta_target)


nn_meta_model = MLPClassifier(hidden_layer_sizes=(200,100,100,50,), activation='relu', solver='adam', max_iter=200, random_state=42)
nn_meta_model.fit(meta_features, meta_target)



MLPClassifier(hidden_layer_sizes=(200, 100, 100, 50), random_state=42)

# Tình profit của các tháng gần đây nhất

In [41]:
data_make_test = data[data['Date'] > time_split].copy()
profit_all = {}
predict_all = {}
for i in range(time_test):
    profit_model =  {}
    predict_model = {}

    time_skips = i
    print('-----------Test tháng thứ ' +str(i+1))
    test_months_ql = data_make_test['Date'].min() + pd.DateOffset(months=0)
    end_months = data_make_test['Date'].min() + pd.DateOffset(months=time_skips+1)
    test_data = data_make_test[(data_make_test['Date'] > test_months_ql) & (data_make_test['Date']<= end_months)].copy()
    X_test = pd.DataFrame(scaler.transform(test_data.drop(['Date'],axis=1)), columns=test_data.drop(['Date'],axis=1).columns)

    X_test['Date'] = test_data['Date'].to_numpy()

    X_test.set_index('Date', inplace=True)

    prices = X_test['CLOSE'].copy().to_numpy()

    meta_features_test = np.zeros((X_test.shape[0], len(base_models)))

    for i, model in enumerate(base_models):
        y_pred_test = model.predict(X_test)
        if isinstance(model, XGBClassifier):
            y_pred_test = le.inverse_transform(y_pred_test)
        meta_features_test[:, i] = y_pred_test



    y_pred = LGBMbase.predict(X_test)
    total_profit = calculate_total_profit(y_pred,prices)
    print(y_pred)
    print('--LGB ' + str(total_profit))
    profit_model["LGB"]  = total_profit
    predict_model["LGB"] = y_pred


    y_pred = LGBMbase2.predict(X_test)
    total_profit = calculate_total_profit(y_pred,prices)
    print(y_pred)
    print('--LGB2 '+ str(total_profit))
    profit_model["LGB2"]  = total_profit
    predict_model["LGB2"] = y_pred

    y_pred = RFbase.predict(X_test)
    total_profit = calculate_total_profit(y_pred,prices)
    print(y_pred)
    print('--RF '+ str(total_profit))
    profit_model["RF"]  = total_profit
    predict_model["RF"] = y_pred

    y_pred = xgbbase.predict(X_test)
    y_pred = le.inverse_transform(y_pred)
    total_profit = calculate_total_profit(y_pred,prices)
    print(y_pred)
    print('--XGB '+ str(total_profit))
    profit_model["XGB"]  = total_profit
    predict_model["XGB"] = y_pred

    y_pred_finalcat = catboost_meta_model.predict(meta_features_test)
    y_pred_finalcat  =  y_pred_finalcat.flatten()
    total_profit = calculate_total_profit(y_pred_finalcat,prices)
    print(y_pred_finalcat)
    print('--meta cat '+ str(total_profit))
    profit_model["meta cat"]  = total_profit
    predict_model["meta cat"] = y_pred_finalcat

    y_pred_finalrf = random_forest_meta_model.predict(meta_features_test)
    total_profit = calculate_total_profit(y_pred_finalrf,prices)
    print(y_pred_finalrf)
    print('--meta rf '+ str(total_profit))
    profit_model["meta rf"]  = total_profit
    predict_model["meta rf"] = y_pred_finalrf

    y_pred_finalMLP = nn_meta_model.predict(meta_features_test)
    total_profit = calculate_total_profit(y_pred_finalMLP,prices)
    print(y_pred_finalMLP)
    print('--meta MLP '+ str(total_profit))
    profit_model["meta mlp"]  = total_profit
    predict_model["meta mlp"] = y_pred_finalMLP



    print("---------------------------------------------------------------------------")
    print("\n")
    month_target = "Tháng "+ str(time_skips+ 1)
    profit_all[month_target] = profit_model
    predict_all[month_target] = predict_model


-----------Test tháng thứ 1
[-1 -1 -1 -1 -1 -1  0  0 -1  0 -1 -1 -1  1 -1  1  0  1  1  0  0 -1 -1]
--LGB -0.2185397226001804
[-1  0  0  0 -1 -1 -1  0  0  0  1  1 -1  1 -1  1  1  1  1  0  0  0 -1]
--LGB2 -0.1410933002869723
[ 0  0  0  0 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
--RF -0.07951166024156087
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
--XGB -0.22877203779065391
[ 0  0  0  0  0  0 -1 -1 -1 -1 -1 -1  0  0  0 -1 -1 -1 -1 -1 -1 -1  0]
--meta cat 0.004224350307993507
[ 0  0  0  0  0  0 -1 -1  1 -1  1  1  0  0  0 -1 -1 -1 -1 -1 -1  1  0]
--meta rf 0.08157689816991232
[ 0  0  0  0 -1  0 -1 -1 -1 -1  1  1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1]
--meta MLP -0.03351317911007978
---------------------------------------------------------------------------


-----------Test tháng thứ 2
[-1 -1 -1 -1 -1 -1  0  0 -1  0 -1 -1 -1  1 -1  1  0  1  1  0  0 -1 -1 -1
  0  0  0  0  0  0  0  0  0  0 -1 -1  0  1  0  1 -1 -1]
--LGB -0.09650293592482129
[-1  0  0

In [42]:
profit_cal = X_test.copy()
profit_cal['PREDICTION'] = y_pred  # Add predictions to DataFrame
profit_cal['PROFIT'] = np.nan  # Initialize profit column
profit_cal['Date'] = profit_cal.index  # Assign index (date) to the 'DATE' column

for i in range(1, len(profit_cal)):
    if profit_cal['PREDICTION'].iloc[i-1] == 1:  # BUY signal
        print(f'BUY signal on {profit_cal["Date"].iloc[i]}')  # Display the date of prediction
        profit_cal['PROFIT'].iloc[i] = profit_cal['CLOSE'].iloc[i] - profit_cal['CLOSE'].iloc[i-1]
    elif profit_cal['PREDICTION'].iloc[i-1] == -1:  # SELL signal
        print(f'SELL signal on {profit_cal["Date"].iloc[i]}')  # Display the date of prediction
        profit_cal['PROFIT'].iloc[i] = profit_cal['CLOSE'].iloc[i-1] - profit_cal['CLOSE'].iloc[i]
    else:
        print('Hold')

# Calculate total profit
total_profit = profit_cal['PROFIT'].sum()


SELL signal on 2024-03-13 00:00:00
SELL signal on 2024-03-14 00:00:00
SELL signal on 2024-03-15 00:00:00
SELL signal on 2024-03-18 00:00:00
SELL signal on 2024-03-19 00:00:00
SELL signal on 2024-03-20 00:00:00
SELL signal on 2024-03-21 00:00:00
SELL signal on 2024-03-22 00:00:00
SELL signal on 2024-03-25 00:00:00
SELL signal on 2024-03-26 00:00:00
SELL signal on 2024-03-27 00:00:00
SELL signal on 2024-03-28 00:00:00
SELL signal on 2024-03-29 00:00:00
BUY signal on 2024-04-01 00:00:00
SELL signal on 2024-04-02 00:00:00
SELL signal on 2024-04-03 00:00:00
SELL signal on 2024-04-04 00:00:00
SELL signal on 2024-04-05 00:00:00
SELL signal on 2024-04-08 00:00:00
SELL signal on 2024-04-09 00:00:00
SELL signal on 2024-04-10 00:00:00
SELL signal on 2024-04-11 00:00:00
SELL signal on 2024-04-12 00:00:00
SELL signal on 2024-04-15 00:00:00
SELL signal on 2024-04-16 00:00:00
SELL signal on 2024-04-17 00:00:00
SELL signal on 2024-04-18 00:00:00
Hold
Hold
Hold
Hold
Hold
Hold
Hold
SELL signal on 2024-0

<ipython-input-42-1a0e35b093fd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  profit_cal['PROFIT'].iloc[i] = profit_cal['CLOSE'].iloc[i-1] - profit_cal['CLOSE'].iloc[i]
<ipython-input-42-1a0e35b093fd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  profit_cal['PROFIT'].iloc[i] = profit_cal['CLOSE'].iloc[i-1] - profit_cal['CLOSE'].iloc[i]
<ipython-input-42-1a0e35b093fd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  profit_cal[

In [44]:
print("Lợi nhuận từng ngày:")
print(profit_cal[['Date', 'PROFIT']])

print("Lợi nhuận tổng cộng:", total_profit)

Lợi nhuận từng ngày:
                 Date    PROFIT
Date                           
2024-03-12 2024-03-12       NaN
2024-03-13 2024-03-13 -0.022248
2024-03-14 2024-03-14  0.008073
2024-03-15 2024-03-15 -0.046562
2024-03-18 2024-03-18 -0.011265
2024-03-19 2024-03-19 -0.092185
2024-03-20 2024-03-20 -0.085895
2024-03-21 2024-03-21  0.004975
2024-03-22 2024-03-22  0.072565
2024-03-25 2024-03-25 -0.015114
2024-03-26 2024-03-26 -0.012485
2024-03-27 2024-03-27  0.009012
2024-03-28 2024-03-28 -0.029852
2024-03-29 2024-03-29  0.008824
2024-04-01 2024-04-01  0.004318
2024-04-02 2024-04-02  0.003943
2024-04-03 2024-04-03 -0.041117
2024-04-04 2024-04-04  0.036799
2024-04-05 2024-04-05  0.016616
2024-04-08 2024-04-08 -0.028632
2024-04-09 2024-04-09  0.003473
2024-04-10 2024-04-10 -0.009294
2024-04-11 2024-04-11 -0.002722
2024-04-12 2024-04-12  0.061676
2024-04-15 2024-04-15 -0.053696
2024-04-16 2024-04-16  0.000375
2024-04-17 2024-04-17 -0.019526
2024-04-18 2024-04-18 -0.022154
2024-04-19 2024-04-